In [42]:
from qiskit import *
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor

In [43]:
# Note: Use larger num_shots in evaluate_circ() than the bit size you need.
# User is supposed to verify by themselves if the random bit num length 
# returned with get_random() meets their minimum length requirements. Number
# returned is going to be <num_shots passed in to evaluate_circ() due to noise
# in real hardware and uncertainty of real measurements. This is not true for
# simulate_circ() as results here are 'idealized' and the condition
# num_shots == len(get_random()) will always hold.
#
# To use, first call init_x_circ() to initialize the circuit with 3 entangled qubits. 
# 'x' can be either 'num' or 'key', depending on if you want to generate a random
# float or bit-string, respectively.
#
# If you choose to run in a simulation (aer-backend), simulate_circ() accepts
# the number of shots requested (corresponding to final length of the
# randomly generated sequence of bits) and a flag (True if want to generate a
# float, false if bit-string). Otherwise, call load_provider() to load
# the available real-hardware backends from IBM-Q, which assumes assumes account 
# token has been already saved, and then evaluate_circ() (accepts the same parameters). 
# 
# Note: you must have instantiated the respective circuit for the requested operation
# prior to executing it.
#
# Once the simulation/request-to-hardware is ran, call get_random() to
# have the random bit sequence returned.

In [44]:
class QBitGen:
    def __init__(self):
        self.__random_num_gen = 0     # Randomly generated float
        self.__random_key_gen = ""    # Randomly generated key
        self.__qrn_key_circ = None    # For generating a random sequence of bits
        self.__qrn_num_circ = None    # For generating a random float 0 < x < 1
        self.__provider = None        # Must be initialized to run on real hardware
        self.__failed_key_results = False # Flag, if set then last attempt at getting key failed
        self.__failed_num_results = False # Flag, if set then last attempt at getting float failed

    # Setup the circuit for random bit sequence
    def init_key_circ(self):
        qcirc = QuantumCircuit(3, 3)

        # Create a state of 3 entangled qbits
        qcirc.h(2)
        qcirc.cx(2, 0)
        qcirc.barrier()

        qcirc.h(1)
        qcirc.cx(1, 0)
        qcirc.barrier()

        # Perform measurements in z-basis
        qcirc.measure(0, 0)
        qcirc.measure(1, 1)
        qcirc.measure(2, 2)

        self.__qrn_key_circ = qcirc
    
    # Setup the circuit for random float
    def init_num_circ(self):
        qcirc = QuantumCircuit(1, 1)
        
        # prepare |+> state
        qcirc.h(0)
        qcirc.barrier()
        
        # Perform measurements in z-basis
        qcirc.measure(0, 0)
        
        self.__qrn_num_circ = qcirc

    # Run the circuit in simulator, pass gen_float flag as True 
    # if want to generate a float 0 < x < 1, otherwise may omit
    def simulate_circ(self, num_shots=1024, gen_float=False):        
        qcirc = None
        
        # Evaluate user request
        if gen_float:
            if self.__qrn_num_circ == None:
                print("Initialize the num circuit first")
                return
            qcirc = self.__qrn_num_circ

        else:
            if self.__qrn_key_circ == None:
                print("Initialize the key circuit first")
                return
            qcirc = self.__qrn_key_circ
        
        assert(qcirc != None)
        
        aer_sim = Aer.get_backend('aer_simulator')
        qobj = assemble(qcirc, shots = num_shots, memory=True)
        result = aer_sim.run(qobj).result().get_memory()

        # Evaluate if float or key op
        if gen_float:
            float_result = 0
            for i in range(len(result)):
                float_result += int(result[i])/(pow(2,i+1))
            if float(float_result) < 0 or float(float_result) > 1:
                self.__failed_num_results = True
                return
            
            self.__random_num_gen = float_result
            self.__failed_num_results = False
            
        else:
            # Verify XOR-rule for obtained results
            random_gen = ""
        
            for measurement in result:
                if int(measurement[0]) ^ int(measurement[1]) != int(measurement[2]):
                    print("Bad result found: " + measurement)
                    self.__failed_key_results = True
                    return

                # Append the measurement
                random_gen += measurement[0]

            self.__random_key_gen = random_gen
            self.__failed_key_results = False
    
    # Load the IBMQ account
    def load_provider(self):
        try:
            IBMQ.load_account()
            self.__provider = IBMQ.get_provider(hub='ibm-q')
        except IBMQProviderError:
            print("Could not fetch providers")
    
    # Run on the real hardware
    def evaluate_circ(self, num_shots=100, gen_float=False):

        if self.__provider == None:
            print("Load the available IBMQ providers first")
            return
        
        qcirc = None
        
        # Evaluate user request
        if gen_float:
            if self.__qrn_num_circ == None:
                print("Initialize the num circuit first")
                return
            qcirc = self.__qrn_num_circ

        else:
            if self.__qrn_key_circ == None:
                print("Initialize the key circuit first")
                return
            qcirc = self.__qrn_key_circ
        
        assert(qcirc != None)
        assert(num_shots > 0)

        # Run the job on the least busy backend and store the results
        backend = least_busy(self.__provider.backends(filters=lambda b: b.configuration().n_qubits == 5 and
                                   not b.configuration().simulator and b.status().operational==True))

        job = backend.run(transpile(qcirc, backend, optimization_level=3), 
                          memory=True, meas_level=2, shots=num_shots)
        job_monitor(job)
        result = job.result().get_memory()

        # Evaluate if float or key op
        if gen_float:
            float_result = 0
            for i in range(len(result)):
                float_result += int(result[i])/(pow(2,i+1))
            if float(float_result) < 0 or float(float_result) > 1:
                self.__failed_num_results = True
                return
            
            self.__random_num_gen = float_result
            self.__failed_num_results = False
            
        else:
            # Verify XOR-rule for obtained results
            random_gen = ""
            for measurement in result:
                if int(measurement[0]) ^ int(measurement[1]) != int(measurement[2]):
                    print("Bad result found: " + measurement)
                    continue

                # Append the measurement
                random_gen += measurement[0]
            
            if random_gen == "":
                self.__failed_key_results = True
                return
            
            self.__random_key_gen = random_gen
            self.__failed_key_results = False
        
    # Return the random, generated sequence, or "" if error
    def get_random(self, get_float=False)->str:
        if get_float:
            if self.__failed_num_results == False:
                return self.__random_num_gen
            print("Could not fetch the float-string, try to generate it again")
        else:
            if self.__random_key_gen != "" and self.__failed_key_results == False:
                return self.__random_key_gen
            print("Could not fetch the bit-string, try to generate it again")